In [1]:
import wandb
import matplotlib.pyplot as plt
import scipy
import numpy as np
import json

plt.rcParams["axes.grid"] = False

sys.path.append('..')

In [2]:
language = 'french'

sys.argv = [
"--device cuda",
"--data-folder", "..\\data",
"--seed", "13",
"--configuration", "char-to-char-encoder-decoder",
"--language", language,
"--challenge", "post-ocr-correction"]

In [3]:
# Configure container:
from dependency_injection.ioc_container import IocContainer

container = IocContainer()

In [4]:
entity = 'eval-historical-texts'
project = 'post-ocr-correction'

In [5]:
def get_wandb_runs():
    api = wandb.Api()
    runs = api.runs(path=f'{entity}/{project}', filters={
        'createdAt': {
            '$gt': '20200621000000'
        },
        'config.language': language,
    })

    return runs

In [6]:
runs = get_wandb_runs()
print(f'Loaded {len(runs)} runs')

Loaded 8 runs


In [7]:
pretrained_model_type_key = 'pretrained_model_type'
fine_tune_key = 'fine_tune'

for i, run in enumerate(runs):
    if len(run.config.keys()) == 0:# or fine_tune_key in run.config.keys() or pretrained_model_type_key in run.config.keys():
        continue

    include_pretrained_model = run.config['include_pretrained_model']
    include_fasttext_model = run.config['include_fasttext_model']
    fine_tune_pretrained = run.config['fine_tune_pretrained']
    fine_tune_after_convergence = run.config['fine_tune_after_convergence']

    pretr_type = 'none'
    if include_pretrained_model and include_fasttext_model:
        pretr_type = 'both'
    elif include_pretrained_model:
        pretr_type = 'bert'
    elif include_fasttext_model:
        pretr_type = 'fast-text'

    fine_tune = (fine_tune_pretrained or fine_tune_after_convergence)

    run.config.update({
        'pretrained_model_type': pretr_type,
        'fine_tune': fine_tune
    })

    run.update()